<a href="https://colab.research.google.com/github/pumkinni/book_study/blob/master/%ED%98%BC%EA%B3%B5_%EB%A8%B8_%EB%94%A5_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치
> 검증 세트가 필요한 이유를 이해하고 교차 검증에 대해 배움

> 그리드 서치와 랜덤 서치를 이용해 최적의 성능을 내는 하이퍼파라미터를 찾음

#### 검증 세트
- 테스트 세트를 사용하지 않기 위해 훈련 세트를 나눈 데이터

- ex) 훈련세트(60%), 검증세트(20%), 테스트세트(20%)

#### 훈련 방법
1. 훈련 세트에서 모델 훈련 후 검증 세트로 모델 평가
2. 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델 선택
3. 이 매개변수를 사용해 전체 훈련 데이터(훈련 세트 + 검증 세트)에서 모델을 다시 훈련
4. 테스트 세트에서 최종 점수 평가

In [ ]:
# 데이터 불러오기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_data')

In [ ]:
# 입력 데이터와 정답 데이터 분리
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
wine.head()
data, target

(array([[ 9.4 ,  1.9 ,  3.51],
        [ 9.8 ,  2.6 ,  3.2 ],
        [ 9.8 ,  2.3 ,  3.26],
        ...,
        [ 9.4 ,  1.2 ,  2.99],
        [12.8 ,  1.1 ,  3.34],
        [11.8 ,  0.8 ,  3.26]]), array([0., 0., 0., ..., 1., 1., 1.]))

In [ ]:
# 테스트 데이터와 훈련 데이터 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [ ]:
# 훈련데이터를 훈련데이터와 검증데이터로 분리
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state= 42)
sub_input.shape, val_input.shape

((4157, 3), (1040, 3))

In [ ]:
# 결정트리분류로 훈련 후 정확도 확인
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state= 42)
dt.fit(sub_input, sub_target)
dt.score(sub_input, sub_target), dt.score(val_input, val_target)

(0.9971133028626413, 0.864423076923077)

#### 교차검증
- 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복

- k-폴드 교차 검증 : 훈련 세트를 k 부분으로 나눠서 교차 검증 을 수행하는 것
  >  ex) 3-폴드 교차 검증
      훈련, 훈련, 검증
      훈련, 검증, 훈련  ---> 검증 점수 평균
      검증, 훈련, 훈련
-교차 검증 함수 : cross_validate(모델 객체, 훈련_input, 훈련_target)
  > fit_time(모델을 훈련하는 시간), score_time(모델을 검증하는 시간), test_score(검증 폴드의 점수) 키를 가진 딕셔너리를 반환

  > 기본 5-폴드 교차 검증

In [ ]:
# 교차검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.01081634, 0.01015735, 0.00819206, 0.00762248, 0.00727725]),
 'score_time': array([0.00145531, 0.00154018, 0.00090623, 0.00086522, 0.00077367]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [ ]:
# 테스트 스코어의 평균값 확인
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# 교차 검즘시 훈련세트를 섞을려면 분할기를 지정
# 사이킷런의 분할기는 교차 검증 시 폴드를 어떻게 나눌지 결정
# 회귀 모델 : KFold, 분류 모델 : StratifiedKFold
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

In [ ]:
# 훈련세트를 섞은 후 10-폴드 교차 검증을 수행
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
np.mean(scores['test_score'])

0.8574181117533719

#### 하이퍼파라미터 튜닝
- 그리드 서치(GridSearchCV) : 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행


In [ ]:
# 탐색할 매개변수 지정(min_impurity_decrease(불순도 감소 최소량) 매개변수의 최적값 찾기)
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
# 그리드 서치 객체 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
# n_jobs : '병렬 실행에 사용할 CPU 코어 수 지정
#'기본값 = 1'(-1 : 시스템에 있는 모든 코어를 사용)

In [ ]:
# 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 
# 전체 훈련 세트에서 자동으로 다시 모델을 훈련 후 gs객체의 best_estimator_ 속성에 저장
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
dt.score(train_input, train_target), dt.score(test_input, test_target)

NameError: ignored

In [ ]:
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [ ]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score'에 저장
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

더 복잡한 매개변수 조합

In [ ]:
# 탐색할 매개변수 지정
# min_umpurity_decrease(노드를 분할하기 위한 불순도 감소 최소량 지정), max_depth(트리의 깊이), min_samples_split(노드를 나누기 위한 최소 샘플 수)
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
          }

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
# 최상의 매개변수 조합 확인
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [ ]:
# 최상의 교차 점수 확인
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

#### 랜덤 서치
> 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 때 사용

> 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달

In [ ]:
# 싸이파이(scipy) - 적분, 보간, 선형대수, 확률 등을 포함한 수치 계산 전용 라이브러리
from scipy.stats import uniform, randint

In [ ]:
# randint : 랜덤하게 정수값 뽑기
rgen = randint(0, 10)
rgen.rvs(10)

array([0, 0, 2, 5, 5, 9, 7, 2, 1, 0])

In [ ]:
np.unique(rgen.rvs(1000), return_counts= True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([102, 116,  89,  99,  92,  93, 104,  95, 103, 107]))

In [ ]:
# uniform : 실숫값 뽑기
ugen = uniform(0,1)
ugen.rvs(10)

array([0.52688575, 0.39582782, 0.50311582, 0.64521245, 0.37452119,
       0.70081294, 0.46143909, 0.22448065, 0.20023845, 0.01764448])

In [ ]:
# 탐색할 매개변수 범위 지정
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25) # 리프 노드가 되기 위한 최소 샘플의 개수
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state=42) # n_ier = "샘플링 횟수"
gs.fit(train_input, train_target)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
35 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 254, in fit
    % self.min_samples_split
ValueError: min_samples_split must be an integer greater than 1 or a float 

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5c9488d710>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5c9488d390>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5c93e93c10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5c9492d4d0>},
                   random_state=42)

In [ ]:
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 12}

In [ ]:
gs.cv_results_['mean_test_score']

array([0.86511513, 0.86261235, 0.86838528, 0.86607796, 0.86376731,
              nan, 0.86280503, 0.86280484, 0.86357592, 0.86357555,
       0.86280503, 0.8626142 , 0.86472977, 0.86954283, 0.86203543,
       0.86761827, 0.86222884, 0.86473033, 0.86877082, 0.86184423,
       0.86165174, 0.86511494, 0.8626142 , 0.86203543, 0.86511476,
       0.86607722, 0.86222773, 0.86684682,        nan, 0.86338436,
       0.8629977 , 0.86242171, 0.86184478, 0.86165211, 0.86049808,
       0.86530706, 0.86280521, 0.8681952 , 0.86203524, 0.86318983,
       0.86780947,        nan, 0.86126694, 0.86934867,        nan,
       0.86530743, 0.86434497, 0.86415303, 0.86838602, 0.86530688,
       0.86145813,        nan, 0.8618446 , 0.86145961, 0.86338454,
       0.86530632, 0.86242152, 0.8645382 , 0.86203543, 0.86376916,
       0.86473051, 0.86184275, 0.86338454, 0.86242004, 0.86107481,
       0.86203654, 0.86184478, 0.86434552, 0.86184478, 0.86299696,
       0.86242023, 0.86338417, 0.86338269,        nan, 0.86415

In [ ]:
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.86